# Importing the Google Trends API and Pandas

In [1]:
from pytrends.request import TrendReq
import pandas as pd

# Reading an Csv and Selecting a Distinct List of Artists

In [ ]:
dataframe = pd.read_csv('Daily Data.csv')
dataframe.head()
artists = dataframe.primary_act_Name.unique()


In [2]:
artists = ['Dababy','David Foster','Taylor Swift',"Lil Uzi Vert",'Playboi Carti']

# Passing each Artist into the Google Trend Api

In [3]:
import time
import math
trendingartists = pd.DataFrame()
trendpct=[]
for art in artists:
    #Intializing pytrends
    pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=5, backoff_factor=0.35)
    kw_list = [art]
    z=pytrends.build_payload(kw_list, cat=0, timeframe="today 3-m", geo='', gprop='')
    df = pytrends.interest_over_time()
    
    
    #df=df.rename({art: 'Trend Percentage'},axis='columns')
    series=df[art]
    mean = series.mean()
    df = series.reset_index()## limits columns to just artist.
    df["Trend Percentage"]=df[art]
   # del df[art]
    
    last_ten_days = df.iloc[-10:] ##limits to the last ten days
    
   

    ten_days_vs_3month_avg=last_ten_days[last_ten_days[art]>mean *2] ##Checking if anyone in the last ten days went above 3 month average
    nadrop=ten_days_vs_3month_avg.dropna() ## Dropping NAN Values
    nadrop['Artist'] = art
    
    #nadrop =nadrop.reset_index() ## Making Date a column so I can use it later in for loop
    #print(nadrop)
     
    
            

    trendpct.append(nadrop) ## Adding all of them to list
for df in trendpct: 
    if df.date.count() >=1: ## Sorting through new list and only adding ones that Return data hence have trended
        #trendingartists.append(df)
        trendingartists = pd.concat([trendingartists,df], sort =False)
        finaltrendingartists=trendingartists[['Artist','date','Trend Percentage']]
        finaltrendingartists=finaltrendingartists.rename({'date':'Date'},axis='columns')
        finaltrendingartists=finaltrendingartists.set_index('Artist')
        del finaltrendingartists.index.name
        finaltrendingartistshtml = finaltrendingartists.to_html()
        



import requests
def send_simple_message():
	return requests.post(
        'https://api.mailgun.net/v3/sandbox3106a32496f34ac4a3a6ebf51a5de903.mailgun.org/messages',
		auth=("api", "dffb6cd15f7e041544f1d63271431707-46ac6b00-c68f626b"),
		data={"from": "Google Trend Checker <mailgun@andbox3106a32496f34ac4a3a6ebf51a5de903.mailgun.org>",
			"to": ["rhysnjennings@gmail.com"], ## Feel free to change email
			"subject": "The Latest trending Artists; Spikes above 2x their average trend",
             "html":"These artists are trending for the past 10 days. Keep an eye on their ticket prices " + finaltrendingartistshtml})
send_simple_message()

<Response [200]>